In [52]:
from matplotlib import pyplot as plt
import numpy as np
import time

class ConnectFour:
    def __init__(self, time_limit=5):
        self.board = self.init_board()
        self.current_player = 1  
        self.time_limit = time_limit  
    @staticmethod
    def init_board():
        return [[0] * 6 for _ in range(7)]

    def drop_disc(self, column):
        for i in range(5, -1, -1):
            if self.board[column][i] == 0:
                self.board[column][i] = self.current_player
                return True
        return False


    def check_winner(self):
        
        for col in range(7):
            for row in range(3):
                if (self.board[col][row] == self.current_player and 
                    self.board[col][row + 1] == self.current_player and
                    self.board[col][row + 2] == self.current_player and
                    self.board[col][row + 3] == self.current_player):
                    return self.current_player
        
        
        for row in range(6):
            for col in range(4):
                if (self.board[col][row] == self.current_player and
                    self.board[col + 1][row] == self.current_player and
                    self.board[col + 2][row] == self.current_player and
                    self.board[col + 3][row] == self.current_player):
                    return self.current_player
        
       
        for col in range(4):
            for row in range(3):
                if (self.board[col][row] == self.current_player and
                    self.board[col + 1][row + 1] == self.current_player and
                    self.board[col + 2][row + 2] == self.current_player and
                    self.board[col + 3][row + 3] == self.current_player):
                    return self.current_player

      
        for col in range(4):
            for row in range(3, 6):
                if (self.board[col][row] == self.current_player and
                    self.board[col + 1][row - 1] == self.current_player and
                    self.board[col + 2][row - 2] == self.current_player and
                    self.board[col + 3][row - 3] == self.current_player):
                    return self.current_player

        return None  
    
    def make_move(self, move_func):
        start_time = time.time()
        column = move_func(self.board)
        end_time=time.time()
        process_time=end_time-start_time
        if process_time> self.time_limit:
            print(f" Time limit {self.current_player}")
            random_player=load_player_module('RandomMoveFunction.py')
            column = random_player.make_move(self.board)
        
        self.drop_disc(column)
        winner = self.check_winner()
        if winner:
            return winner, process_time
        self.current_player = 3 - self.current_player  
        return None, process_time  

In [46]:
import importlib.util

def load_player_module(module_path):
    spec = importlib.util.spec_from_file_location("module.name", module_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

def simulate_game(player1_path, player2_path, time_limit):
    game = ConnectFour(time_limit)
    boards=[]
    player1 = load_player_module(player1_path)
    player2 = load_player_module(player2_path)
    players = {1: player1, 2: player2}
    times={1:[], 2:[]}
    
    while True:
        result, process_time = game.make_move(players[game.current_player].make_move)
        times[3-game.current_player]=process_time
        boards.append(np.transpose(game.board))
        if result:
            return result, boards , times

In [36]:
import tkinter as tk
from tkinter import messagebox

class GameBoard(tk.Frame):
    def __init__(self, master, board_states, message, size=50):
        """ Initialize the board with a list of board states and an end message """
        super().__init__(master)
        self.master = master
        self.board_states = board_states
        self.end_message = message
        self.current_state = 0
        self.rows = len(board_states[0])
        self.cols = len(board_states[0][0])
        self.size = size
        self.cells = {}
        self.grid()

        # Initialize GUI components for the board
        self.init_board()
        self.update_board()  # Display the initial state

    def init_board(self):
        """ Initialize the GUI representation of the board """
        for row in range(self.rows):
            for col in range(self.cols):
                frame = tk.Frame(self, width=self.size, height=self.size,
                                 borderwidth=1, relief="solid")
                frame.grid(row=row, column=col, padx=0, pady=0)
                self.cells[(row, col)] = frame

    def update_board(self):
        """ Update the GUI based on the current board state """
        board = self.board_states[self.current_state]
        for row in range(self.rows):
            for col in range(self.cols):
                color = 'white' if board[row][col] == 0 else 'blue' if board[row][col] == 1 else 'red'
                self.cells[(row, col)].configure(background=color)

        self.current_state += 1  # Move to the next state
        if self.current_state < len(self.board_states):  # Check if there are more states to show
            self.after(1000, self.update_board)  # Schedule the next update
        else:
            self.after(1000, self.show_end_message)  # Show end message after the last update

    def show_end_message(self):
        """ Show an end message after the simulation """
        messagebox.showinfo("Simulation Complete", self.end_message)


In [55]:
result, boards, times=simulate_game('RandomMoveFunction.py', 'RandomMoveFunction.py', 2)
root = tk.Tk()
root.eval('tk::PlaceWindow . center')
root.title('Connect-4')
game = GameBoard(root, boards, f"Winner is Player {result} \n Time_player1={np.mean(times[1])}\nTime_player2={np.mean(times[2])}")
root.mainloop()